### **Group Id - 1801cs36_1801cs37**
*   1801CS36 - Mahesh Babu
*   1801CS37 - Sriram Pingali

In [1]:
from queue import PriorityQueue
from dataclasses import dataclass
import random
import time

optimal_path = []
MAX_ITERATIONS = 50000

Node Class

In [2]:
@dataclass(order=True)
class Node():
    """Node class :-
        state: board configuration(a list)
        depth: Node depth - g(n)
        h1: Number of tiles displaced from their destined position
        h2: Sum of Manhattan distance of each tile from the goal position
        parent: parent of the current node 
    """
    def __init__(self, state, depth):
        self.state = state
        self.depth = depth
        self.h1 = None
        self.h2 = None
        self.parent = None
        self.priority = random.randrange(100)

    def calc_heuristic(self, goal_state):
        self.h1 = h1(self.state, goal_state)
        self.h2 = h2(self.state, goal_state)
        self.greedy_dist = self.h1
        self.a_star_dist = self.depth + self.h2

    def back_track(self):
        current = self
        count = 0
        optimal_path.clear()
        while (current != None):
            bs = board_state(current.state)
            optimal_path.append(current.state)
            count += 1
            current = current.parent
        return count

Heuristics

In [3]:
def h1(current_state, goal_state):
    """Number of tiles displaced from their destined position."""
    cost=0
    # print(current_state, goal_state)
    for i in range(len(current_state)):
        if current_state[i]!=goal_state[i]:
            cost+=1
    return cost

def h2(current_state, goal_state): 
    """Sum of Manhattan distance of each tile from the goal position."""
    cost=0
    final_position = cordinates(goal_state)
    temp=board_state(current_state)
    for i in range(3):
        for j in range(3):
            t=temp[i][j]
            xf, yf = final_position[t]
            cost += abs(xf-i)+abs(yf-j)
    return cost

Utility Functions

In [4]:
def board_state(state):
    """Return 2-d matrix representation"""
    i=0
    temp=[([0]*3) for j in range(3)]
    for row in range(3):
        for col in range(3):
            temp[row][col]=state[i]
            i+=1
    return temp

def display_board(state):
    """Print the board"""
    print("-------------")
    print("| %i | %i | %i |" % (state[0], state[1], state[2]))
    print("-------------")
    print("| %i | %i | %i |" % (state[3], state[4], state[5]))
    print("-------------")
    print("| %i | %i | %i |" % (state[6], state[7], state[8]))
    print("-------------")

def display_start_goal(start, goal):
    """Print start and goal states"""
    print("START STATE               GOAL STATE")
    print("-------------             -------------")
    print("| %i | %i | %i |             | %i | %i | %i |" % (start[0], start[1], start[2], goal[0], goal[1], goal[2]))
    print("-------------             -------------")
    print("| %i | %i | %i |    ---->    | %i | %i | %i |" % (start[3], start[4], start[5], goal[3], goal[4], goal[5]))
    print("-------------             -------------")
    print("| %i | %i | %i |             | %i | %i | %i |" % (start[6], start[7], start[8], goal[6], goal[7], goal[8]))
    print("-------------             -------------")

def cordinates(state):
    """Return position coordinates in the goal state"""
    cords = [None] * 9
    for index, i  in enumerate(state):
        cords[i] = (index // 3, index % 3)
    return cords

# move the blank title up on the board
def move_up(state):
    new_state=state[:]
    index = new_state.index(0)
    if index not in [0,1,2]:
        temp = new_state[index-3]
        new_state[index-3]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title down on the board
def move_down(state):
    new_state=state[:]
    index=new_state.index(0)
    if index not in [6,7,8]:
        temp = new_state[index+3]
        new_state[index+3]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title left on the board
def move_left(state):
    new_state = state[:]
    index = new_state.index(0)
    if index not in [0,3,6]:
        temp = new_state[index-1]
        new_state[index-1]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title right on the board
def move_right(state):
    new_state = state[:]
    index = new_state.index(0)
    if index not in [2,5,8]:
        temp = new_state[index+1]
        new_state[index+1] = new_state[index]
        new_state[index]=temp
    return new_state

In [5]:
def expansion(state):
    """Expand nodes along the children"""
    expanded_nodes = []
    expanded_nodes.append(move_up(state))       # moving up
    expanded_nodes.append(move_down(state))     # moving down
    expanded_nodes.append(move_left(state))     # moving left
    expanded_nodes.append(move_right(state))    # moving right
    expanded_nodes = [x for x in expanded_nodes if x]
    return(expanded_nodes)

Greedy Best First Search

In [6]:
def bfs(start_state, goal_state):
    """Cost function: f(n) = h(n)"""
    iterations = 0
    open = PriorityQueue()
    closed = []

    start_node = Node(start_state, 0)
    start_node.calc_heuristic(goal_state)

    open.put((start_node.greedy_dist, start_node))
    while (not open.empty()):
        iterations+=1
        if iterations>MAX_ITERATIONS:
            print("Failure: No solution found")
            print("Total number of states explored:\n", len(closed))
            return None,None
        cost, parent = open.get()
        closed.append(parent.state)
        if (parent.state == goal_state):
            return (parent, len(closed))
        for i in expansion(parent.state):

            temp_node = Node(i, parent.depth + 1)
            temp_node.parent = parent

            if (temp_node.state == goal_state):
                return (temp_node, len(closed))
            elif (temp_node.state in closed):
                continue
            
            temp_node.calc_heuristic(goal_state)
            open.put((temp_node.greedy_dist, temp_node))

A* Search

In [7]:
def a_star(start_state, goal_state):
    """Cost function: f(n) = g(n) + h(n)"""
    open = PriorityQueue()
    closed = []
    iterations = 0
    start_node = Node(start_state, 0)
    start_node.calc_heuristic(goal_state)

    open.put((start_node.a_star_dist, start_node))
    while (not open.empty()):
        if iterations>MAX_ITERATIONS:
            print("Failure! No solution found")
            print("Total number of states explored:", len(closed))
            return None,None
        _, parent = open.get()
        closed.append(parent.state)
        if (parent.state == goal_state):
            return (parent, len(closed))
        for i in expansion(parent.state):

            temp_node = Node(i, parent.depth + 1)
            temp_node.parent = parent

            if (temp_node.state == goal_state):
                return (temp_node, len(closed))
            elif (temp_node.state in closed):
                continue
            
            temp_node.calc_heuristic(goal_state)
            open.put((temp_node.a_star_dist, temp_node))

Driver code

In [8]:
# start_state= [6, 7, 3, 8, 4, 2, 5, 0, 1]
# goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

# start_state= [1, 2, 3, 0, 4, 5, 6, 7, 8]
# goal_state = [7, 3, 2, 5, 4, 8, 1, 0, 6]

start_state= [1, 2, 3, 0, 4, 5, 6, 7, 8]
goal_state = [2, 3, 5, 6, 1, 8, 4, 7, 0]

# start_state= [1, 2, 3, 0, 4, 5, 6, 7, 8]
# goal_state = [1, 2, 3, 4, 5, 0, 6, 7, 8]

# No Solution
# start_state = [6, 7, 3, 8, 4, 2, 1, 0, 5]
# goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

# start_state = [2, 8, 1, 4, 6, 3, 7, 5, 0]
# goal_state = [1, 2, 3, 8, 0, 4, 7, 6, 5]

def main():
    display_start_goal(start_state, goal_state)
    
    print("\nGreedy Best First Search:\n")
    start_clock = time.time()
    node_1, expl = bfs(start_state, goal_state)
    if expl!=None:
        print("Success! Solution found")
        bfs_execution_time = time.time()-start_clock
        print("Total number of states explored:",expl)
        optimal_cost = node_1.back_track()
        print("Total number of states in optimal path:", optimal_cost)
        print("Optimal Path:")
        for state in optimal_path:
            display_board(state)
        print("Optimal path cost:", optimal_cost)
        print("Time taken for execution (Greedy Bfs):", bfs_execution_time)

        print("______________________________________________________\n")
    
    print("A* First Search")
    start_clock = time.time()
    node_1, expl = a_star(start_state, goal_state)
    if expl!=0:
        print("Success! Solution found")
        astar_execution_time = time.time()-start_clock
        print("Total number of states explored:",expl)
        optimal_cost = node_1.back_track()
        print("Total number of states in optimal path:", optimal_cost)
        print("Optimal Path:")
        for state in optimal_path:
            display_board(state)
        print("Optimal path cost:", optimal_cost)
        print("Time taken for execution (A*):", astar_execution_time)

if __name__ == "__main__":
    main()

START STATE               GOAL STATE
-------------             -------------
| 1 | 2 | 3 |             | 2 | 3 | 5 |
-------------             -------------
| 0 | 4 | 5 |    ---->    | 6 | 1 | 8 |
-------------             -------------
| 6 | 7 | 8 |             | 4 | 7 | 0 |
-------------             -------------

Greedy Best First Search:

Success! Solution found
Total number of states explored: 31
Total number of states in optimal path: 12
Optimal Path:
-------------
| 2 | 3 | 5 |
-------------
| 6 | 1 | 8 |
-------------
| 4 | 7 | 0 |
-------------
-------------
| 2 | 3 | 5 |
-------------
| 6 | 1 | 8 |
-------------
| 4 | 0 | 7 |
-------------
-------------
| 2 | 3 | 5 |
-------------
| 6 | 1 | 8 |
-------------
| 0 | 4 | 7 |
-------------
-------------
| 2 | 3 | 5 |
-------------
| 0 | 1 | 8 |
-------------
| 6 | 4 | 7 |
-------------
-------------
| 2 | 3 | 5 |
-------------
| 1 | 0 | 8 |
-------------
| 6 | 4 | 7 |
-------------
-------------
| 2 | 3 | 5 |
-------------
| 1 | 